# **Homework 10 - Adversarial Attack**

## Enviroment & Download

We make use of [pytorchcv](https://pypi.org/project/pytorchcv/) to obtain CIFAR-10 pretrained model, so we need to set up the enviroment first. We also need to download the data (200 images) which we want to attack.

In [1]:
# set up environment
!pip install pytorchcv
!pip install imgaug

# download
# !gdown --id 1t2UFQXr1cr5qLMBK2oN2rY1NDypi9Nyw --output data.zip

# if the above link isn't available, try this one
!wget https://www.dropbox.com/s/lbpypqamqjpt2qz/data.zip
!pip install unzip
# unzip
!unzip ./data.zip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.4/532.4 kB 14.6 MB/s eta 0:00:00
--2023-05-13 09:23:55--  https://www.dropbox.com/s/lbpypqamqjpt2qz/data.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/lbpypqamqjpt2qz/data.zip [following]
--2023-05-13 09:23:55--  https://www.dropbox.com/s/raw/lbpypqamqjpt2qz/data.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc825fbd1cfbc6135e74d84d5fed.dl.dropboxusercontent.com/cd/0/inline/B7_Tfl2cTZ8M04pXotOHXCg0r-HD8p5OtMjw2dky5ia2wNVC6zY4ix6TmDFJxFwf5VSkDfSoIpTGzC9HRUqT4MO4x6WSz1_mJVyy4glmh5a3FMOnoa8aglRydQCpUisDCJW53skJ9LIxrVxeWOjkDMuPwvIOC5c-nMSXvR055Hk09A/file# [following]
--2023-05-13 09:23:56--  https://uc825fbd1cfbc6135e74d84d5fed.dl.dropboxusercontent.com/cd/0/inline/B7_Tfl2cTZ8

In [2]:
!rm ./data.zip

In [3]:
import torch
import torch.nn as nn
from pytorchcv.model_provider import get_model as ptcv_get_model
import random
import numpy as np
import torch.nn.functional as F

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_size = 8

def same_seeds(seed):
	  torch.manual_seed(seed)
	  if torch.cuda.is_available():
		    torch.cuda.manual_seed(seed)
		    torch.cuda.manual_seed_all(seed)
	  np.random.seed(seed)
	  random.seed(seed)
	  torch.backends.cudnn.benchmark = False
	  torch.backends.cudnn.deterministic = True
same_seeds(1999) 

## Global Settings 
#### **[NOTE]**: Don't change the settings here, or your generated image might not meet the constraint.
* $\epsilon$ is fixed to be 8. But on **Data section**, we will first apply transforms on raw pixel value (0-255 scale) **by ToTensor (to 0-1 scale)** and then **Normalize (subtract mean divide std)**. $\epsilon$ should be set to $\frac{8}{255 * std}$ during attack.

* Explaination (optional)
    * Denote the first pixel of original image as $p$, and the first pixel of adversarial image as $a$.
    * The $\epsilon$ constraints tell us $\left| p-a \right| <= 8$.
    * ToTensor() can be seen as a function where $T(x) = x/255$.
    * Normalize() can be seen as a function where $N(x) = (x-mean)/std$ where $mean$ and $std$ are constants.
    * After applying ToTensor() and Normalize() on $p$ and $a$, the constraint becomes $\left| N(T(p))-N(T(a)) \right| = \left| \frac{\frac{p}{255}-mean}{std}-\frac{\frac{a}{255}-mean}{std} \right| = \frac{1}{255 * std} \left| p-a \right| <= \frac{8}{255 * std}.$
    * So, we should set $\epsilon$ to be $\frac{8}{255 * std}$ after ToTensor() and Normalize().

In [4]:
# the mean and std are the calculated statistics from cifar_10 dataset
cifar_10_mean = (0.491, 0.482, 0.447) # mean for the three channels of cifar_10 images
cifar_10_std = (0.202, 0.199, 0.201) # std for the three channels of cifar_10 images

# convert mean and std to 3-dimensional tensors for future operations
mean = torch.tensor(cifar_10_mean).to(device).view(3, 1, 1)
std = torch.tensor(cifar_10_std).to(device).view(3, 1, 1)

epsilon = 8/255/std

In [5]:
root = './data' # directory for storing benign images
# benign images: images which do not contain adversarial perturbations
# adversarial images: images which include adversarial perturbations

## Data

Construct dataset and dataloader from root directory. Note that we store the filename of each image for future usage.

In [6]:
import os
import glob
import shutil
import numpy as np
from PIL import Image
from torchvision.transforms import transforms
from torch.utils.data import Dataset, DataLoader

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(cifar_10_mean, cifar_10_std)
])

class AdvDataset(Dataset):
    def __init__(self, data_dir, transform):
        self.images = []
        self.labels = []
        self.names = []
        '''
        data_dir
        ├── class_dir
        │   ├── class1.png
        │   ├── ...
        │   ├── class20.png
        '''
        for i, class_dir in enumerate(sorted(glob.glob(f'{data_dir}/*'))):
            images = sorted(glob.glob(f'{class_dir}/*'))
            self.images += images
            self.labels += ([i] * len(images))
            self.names += [os.path.relpath(imgs, data_dir) for imgs in images]
        self.transform = transform
    def __getitem__(self, idx):
        image = self.transform(Image.open(self.images[idx]))
        label = self.labels[idx]
        return image, label
    def __getname__(self):
        return self.names
    def __len__(self):
        return len(self.images)

adv_set = AdvDataset(root, transform=transform)
adv_names = adv_set.__getname__()
adv_loader = DataLoader(adv_set, batch_size=batch_size, shuffle=False)

print(f'number of images = {adv_set.__len__()}')

number of images = 200


## Utils -- Benign Images Evaluation

In [7]:
# to evaluate the performance of model on benign images
def epoch_benign(model, loader, loss_fn):
    model.eval()
    train_acc, train_loss = 0.0, 0.0
    for x, y in loader:
        x, y = x.to(device), y.to(device)
        yp = model(x)
        loss = loss_fn(yp, y)
        train_acc += (yp.argmax(dim=1) == y).sum().item()
        train_loss += loss.item() * x.shape[0]
    return train_acc / len(loader.dataset), train_loss / len(loader.dataset)

## Utils -- Attack Algorithm

In [8]:
# perform fgsm attack
def fgsm(model, x, y, loss_fn, epsilon=epsilon):
    x_adv = x.detach().clone() # initialize x_adv as original benign image x
    x_adv.requires_grad = True # need to obtain gradient of x_adv, thus set required grad
    loss = loss_fn(model(x_adv), y) # calculate loss
    loss.backward() # calculate gradient
    # fgsm: use gradient ascent on x_adv to maximize loss
    grad = x_adv.grad.detach()
    x_adv = x_adv + epsilon * grad.sign()
    return x_adv

# alpha and num_iter can be decided by yourself
alpha = 0.8/255/std

def ifgsm(model, x, y, loss_fn, epsilon=epsilon, alpha=alpha, num_iter=20):
    x_adv = x.detach().clone()
    ################ TODO: Medium baseline #######################
    # write a loop with num_iter times
    for i in range(num_iter):
      # TODO: Each iteration, execute fgsm
        x_adv = x.detach().clone() # initialize x_adv as original benign image x
        x_adv.requires_grad = True # need to obtain gradient of x_adv, thus set required grad
        loss = loss_fn(model(x_adv), y) # calculate loss
        loss.backward() # calculate gradient
        # fgsm: use gradient ascent on x_adv to maximize loss
        x_adv = x_adv + epsilon * x_adv.grad.detach().sign()
    return x_adv

def mifgsm(model, x, y, loss_fn, epsilon=epsilon, alpha=alpha, num_iter=20, decay=0.1):
    x_adv = x
    # initialze momentum tensor
    momentum = torch.zeros_like(x).detach().to(device)

    ################ TODO: Strong baseline ####################
    for i in range(num_iter):
      # TODO: Refer to the algorithm of MI-FGSM
      # Calculate the momentum and update
        x_adv = x_adv.detach().clone()
        x_adv.requires_grad = True
        loss = loss_fn(model(x_adv), y) # calculate loss
        loss.backward() # calculate gradient
        grad = x_adv.grad.detach()
        grad = grad / torch.mean(torch.abs(grad), dim=(1,2,3), keepdim=True)
        grad = grad + momentum * decay
        momentum = grad
        x_adv = x_adv + alpha * grad.sign()
        x_adv = torch.max(torch.min(x_adv, x+epsilon), x-epsilon)
    return x_adv

def nifgsm(model, x, y, loss_fn, epsilon=epsilon, alpha=alpha, num_iter=100, decay=1.0):
    x_adv = x
    # initialze momentum tensor
    momentum = torch.zeros_like(x).detach().to(device)
    ################ TODO: Strong baseline ####################
    for i in range(num_iter):
      # TODO: Refer to the algorithm of NI-FGSM
      # Calculate the momentum and update
        x_adv = x_adv.detach().clone()
        x_adv.requires_grad = True
        x_adv = x_adv + decay*alpha*momentum
        # Calculate loss
        loss = loss_fn(model(x_adv), y)

        # Update adversarial images
        grad = torch.autograd.grad(loss, x_adv,
                                   retain_graph=False, create_graph=False)[0]
        grad = decay*momentum + grad / torch.mean(torch.abs(grad), dim=(1,2,3), keepdim=True)
        momentum = grad
        x_adv = x_adv + alpha * grad.sign()
        x_adv = torch.max(torch.min(x_adv, x+epsilon), x-epsilon)
    return x_adv

def snifgsm(model, x, y, loss_fn, epsilon=epsilon, alpha=alpha, num_iter=100, decay=1.0, m=20):
    x_adv = x
    # initialze momentum tensor
    momentum = torch.zeros_like(x).detach().to(device)
    ################ TODO: Strong baseline ####################
    for i in range(num_iter):
      # TODO: Refer to the algorithm of NI-FGSM
      # Calculate the momentum and update
        x_adv = x_adv.detach().clone()
        x_adv.requires_grad = True
        nes_image = x_adv + decay*alpha*momentum
        # Calculate sum the gradients over the scale copies of the input image
        adv_grad = torch.zeros_like(x).detach().to(device)
        for i in torch.arange(m):
            nes_images = nes_image / torch.pow(2, i)
            # Calculate loss
            loss = loss_fn(model(nes_images), y)
            adv_grad += torch.autograd.grad(loss, x_adv,
                                            retain_graph=False, create_graph=False)[0]
        grad = adv_grad / m
        

        # Update adversarial images
        
        grad = decay*momentum + grad / torch.mean(torch.abs(grad), dim=(1,2,3), keepdim=True)
        momentum = grad
        x_adv = x_adv + alpha * grad.sign()
        x_adv = torch.max(torch.min(x_adv, x+epsilon), x-epsilon)
    return x_adv

## Utils -- Attack
* Recall
  * ToTensor() can be seen as a function where $T(x) = x/255$.
  * Normalize() can be seen as a function where $N(x) = (x-mean)/std$ where $mean$ and $std$ are constants.

* Inverse function
  * Inverse Normalize() can be seen as a function where $N^{-1}(x) = x*std+mean$ where $mean$ and $std$ are constants.
  * Inverse ToTensor() can be seen as a function where $T^{-1}(x) = x*255$.

* Special Noted
  * ToTensor() will also convert the image from shape (height, width, channel) to shape (channel, height, width), so we also need to transpose the shape back to original shape.
  * Since our dataloader samples a batch of data, what we need here is to transpose **(batch_size, channel, height, width)** back to **(batch_size, height, width, channel)** using np.transpose.

In [9]:
# perform adversarial attack and generate adversarial examples
def gen_adv_examples(model, loader, attack, loss_fn):
    model.eval()
    adv_names = []
    train_acc, train_loss = 0.0, 0.0
    for i, (x, y) in enumerate(loader):
        x, y = x.to(device), y.to(device)
        x_adv = attack(model, x, y, loss_fn) # obtain adversarial examples
        yp = model(x_adv)
        loss = loss_fn(yp, y)
        train_acc += (yp.argmax(dim=1) == y).sum().item()
        train_loss += loss.item() * x.shape[0]
        # store adversarial examples
        adv_ex = ((x_adv) * std + mean).clamp(0, 1) # to 0-1 scale
        adv_ex = (adv_ex * 255).clamp(0, 255) # 0-255 scale
        adv_ex = adv_ex.detach().cpu().data.numpy().round() # round to remove decimal part
        adv_ex = adv_ex.transpose((0, 2, 3, 1)) # transpose (bs, C, H, W) back to (bs, H, W, C)
        adv_examples = adv_ex if i == 0 else np.r_[adv_examples, adv_ex]
    return adv_examples, train_acc / len(loader.dataset), train_loss / len(loader.dataset)

# create directory which stores adversarial examples
def create_dir(data_dir, adv_dir, adv_examples, adv_names):
    if os.path.exists(adv_dir) is not True:
        _ = shutil.copytree(data_dir, adv_dir)
    for example, name in zip(adv_examples, adv_names):
        im = Image.fromarray(example.astype(np.uint8)) # image pixel value should be unsigned int
        im.save(os.path.join(adv_dir, name))

## Model / Loss Function

Model list is available [here](https://github.com/osmr/imgclsmob/blob/master/pytorch/pytorchcv/model_provider.py). Please select models which has _cifar10 suffix. Other kinds of models are prohibited, and it will be considered to be cheating if you use them. 

Note: Some of the models cannot be accessed/loaded. You can safely skip them since TA's model will not use those kinds of models.

In [10]:
# This function is used to check whether you use models pretrained on cifar10 instead of other datasets
def model_checker(model_name):
  assert ('cifar10' in model_name) and ('cifar100' not in model_name), 'The model selected is not pretrained on cifar10!'

In [11]:
from pytorchcv.model_provider import get_model as ptcv_get_model

model_name = 'resnet110_cifar10'
model_checker(model_name)

model = ptcv_get_model(model_name, pretrained=True).to(device)
loss_fn = nn.CrossEntropyLoss()

benign_acc, benign_loss = epoch_benign(model, adv_loader, loss_fn)
print(f'benign_acc = {benign_acc:.5f}, benign_loss = {benign_loss:.5f}')

benign_acc = 0.95000, benign_loss = 0.22678


## FGSM

In [12]:
# adv_examples, fgsm_acc, fgsm_loss = gen_adv_examples(model, adv_loader, fgsm, loss_fn)
# print(f'fgsm_acc = {fgsm_acc:.5f}, fgsm_loss = {fgsm_loss:.5f}')

# create_dir(root, 'fgsm', adv_examples, adv_names)

# IFGSM

In [13]:
# adv_examples, ifgsm_acc, ifgsm_loss = gen_adv_examples(model, adv_loader, ifgsm, loss_fn)
# print(f'ifgsm_acc = {ifgsm_acc:.5f}, ifgsm_loss = {ifgsm_loss:.5f}')

# create_dir(root, 'ifgsm', adv_examples, adv_names)

# MIFGSM

In [14]:
# adv_examples, mifgsm_acc, mifgsm_loss = gen_adv_examples(model, adv_loader, mifgsm, loss_fn)
# print(f'mifgsm_acc = {mifgsm_acc:.5f}, mifgsm_loss = {mifgsm_loss:.5f}')

# create_dir(root, 'mifgsm', adv_examples, adv_names)

In [15]:
# %cd fgsm
# !tar zcvf ../fgsm.tgz *
# %cd ..
# %cd ifgsm
# !tar zcvf ../ifgsm.tgz *
# %cd ..
# %cd mifgsm
# !tar zcvf ../mifgsm.tgz *
# %cd ..

## Example of Ensemble Attack
* Ensemble multiple models as your proxy model to increase the black-box transferability ([paper](https://arxiv.org/abs/1611.02770))

In [16]:
################ BOSS BASELINE ######################

class ensembleNet(nn.Module):
    def __init__(self, model_names):
        super().__init__()
        self.models = nn.ModuleList([ptcv_get_model(name, pretrained=True) for name in model_names])
        
    def forward(self, x):
        #################### TODO: boss baseline ###################
        ensemble_logits = 0
        for i, m in enumerate(self.models):
            logits = m(x.clone())
            ensemble_logits += logits
        ensemble_logits = ensemble_logits / len(self.models)
        return ensemble_logits
        # TODO: sum up logits from multiple models  
        # return ensemble_logits

* Construct your ensemble model

In [17]:
model_names = [
    'preresnet110_cifar10',
    'sepreresnet110_cifar10',
    'diapreresnet110_cifar10',
    'densenet40_k24_bc_cifar10',
    'densenet100_k12_cifar10',
    'resnet56_cifar10',
    'rir_cifar10',
    'ror3_110_cifar10',
#     'sepreresnet20_cifar10',
#     'sepreresnet56_cifar10',
#     'diaresnet20_cifar10',
#     'diaresnet56_cifar10',
]

for model_num in model_names:
    model = ptcv_get_model(model_num, pretrained=True).to(device)
    for param in model.parameters():
        param.requires_grad_(False)

    bnunign_acc, benign_loss = epoch_benign(model, adv_loader, loss_fn)
    print(f'Model : {model_num} -> benign_acc = {benign_acc:.5f}, benign_loss = {benign_loss:.5f}')

# for model_name in model_names:
#   model_checker(model_name)

ensemble_model = ensembleNet(model_names).to(device)
ensemble_model.eval()
print()

Model : preresnet110_cifar10 -> benign_acc = 0.95000, benign_loss = 0.28506
Model : sepreresnet110_cifar10 -> benign_acc = 0.95000, benign_loss = 0.27381
Model : diapreresnet110_cifar10 -> benign_acc = 0.95000, benign_loss = 0.20748
Model : densenet40_k24_bc_cifar10 -> benign_acc = 0.95000, benign_loss = 0.20855
Model : densenet100_k12_cifar10 -> benign_acc = 0.95000, benign_loss = 0.28858
Model : resnet56_cifar10 -> benign_acc = 0.95000, benign_loss = 0.16539
Model : rir_cifar10 -> benign_acc = 0.95000, benign_loss = 0.14601
Model : ror3_110_cifar10 -> benign_acc = 0.95000, benign_loss = 0.28892



In [18]:
benign_acc, benign_loss = epoch_benign(ensemble_model, adv_loader, loss_fn)
print(f'benign_acc = {benign_acc:.5f}, benign_loss = {benign_loss:.5f}')

benign_acc = 0.95500, benign_loss = 0.12889


In [19]:
def input_diversity(x, resize_rate, diversity_prob):
    img_size = x.shape[-1]
    img_resize = int(img_size * resize_rate)

    if resize_rate < 1:
        img_size = img_resize
        img_resize = x.shape[-1]

    rnd = torch.randint(low=img_size, high=img_resize, size=(1,), dtype=torch.int32)
    rescaled = F.interpolate(x, size=[rnd, rnd], mode='bilinear', align_corners=False)
    h_rem = img_resize - rnd
    w_rem = img_resize - rnd
    pad_top = torch.randint(low=0, high=h_rem.item(), size=(1,), dtype=torch.int32)
    pad_bottom = h_rem - pad_top
    pad_left = torch.randint(low=0, high=w_rem.item(), size=(1,), dtype=torch.int32)
    pad_right = w_rem - pad_left

    padded = F.pad(rescaled, [pad_left.item(), pad_right.item(), pad_top.item(), pad_bottom.item()], value=0)

    return padded if torch.rand(1) < diversity_prob else x


def dmifgsm(model, x, y, loss_fn, epsilon=epsilon, alpha=alpha, num_iter=100, decay=1.0,
            resize_rate=0.9, diversity_prob=0.5):
    x_adv = x
    # initialze momentum tensor
    momentum = torch.zeros_like(x).detach().to(device)

    # write a loop of num_iter to represent the iterative times
    for i in range(num_iter):
        x_adv = x_adv.detach().clone()
        x_adv.requires_grad = True # need to obtain gradient of x_adv, thus set required grad
        loss = loss_fn(model(input_diversity(x_adv, resize_rate, diversity_prob)), y) # calculate loss
#         loss.backward() # calculate gradient
        # Update adversarial images
        grad = torch.autograd.grad(loss, x_adv, retain_graph=False, create_graph=False)[0]
#         grad = x_adv.grad.detach()
        grad = grad / torch.mean(torch.abs(grad), dim=(1,2,3), keepdim=True)
        grad = grad + momentum * decay
        momentum = grad
        x_adv = x_adv + alpha * grad.sign()
        x_adv = torch.max(torch.min(x_adv, x+epsilon), x-epsilon)
    return x_adv

In [20]:
# ATTACK_METHOD=snifgsm

In [21]:
# adv_examples, ifgsm_acc, ifgsm_loss = gen_adv_examples(ensemble_model, adv_loader, ATTACK_METHOD, loss_fn)
# print(f'{ATTACK_METHOD}_acc = {ifgsm_acc:.5f}, {ATTACK_METHOD}_loss = {ifgsm_loss:.5f}')
# ATTACK_METHOD='snifgsm'
# create_dir(root, ATTACK_METHOD, adv_examples, adv_names)

In [22]:
ATTACK_METHOD=dmifgsm
adv_examples, ifgsm_acc, ifgsm_loss = gen_adv_examples(ensemble_model, adv_loader, ATTACK_METHOD, loss_fn)
print(f'{ATTACK_METHOD}_acc = {ifgsm_acc:.5f}, {ATTACK_METHOD}_loss = {ifgsm_loss:.5f}')
ATTACK_METHOD='dmifgsm'
create_dir(root, ATTACK_METHOD, adv_examples, adv_names)
%cd dmifgsm
!tar zcvf ../dmifgsm.tgz *
%cd ..

<function dmifgsm at 0x7c6931df16c0>_acc = 0.00000, <function dmifgsm at 0x7c6931df16c0>_loss = 15.84042
/kaggle/working/dmifgsm
airplane/
airplane/airplane3.png
airplane/airplane16.png
airplane/airplane11.png
airplane/airplane18.png
airplane/airplane17.png
airplane/airplane9.png
airplane/airplane15.png
airplane/airplane20.png
airplane/airplane13.png
airplane/airplane10.png
airplane/airplane5.png
airplane/airplane12.png
airplane/airplane7.png
airplane/airplane4.png
airplane/airplane19.png
airplane/airplane1.png
airplane/airplane14.png
airplane/airplane8.png
airplane/airplane2.png
airplane/airplane6.png
automobile/
automobile/automobile2.png
automobile/automobile9.png
automobile/automobile10.png
automobile/automobile6.png
automobile/automobile3.png
automobile/automobile8.png
automobile/automobile4.png
automobile/automobile13.png
automobile/automobile7.png
automobile/automobile5.png
automobile/automobile17.png
automobile/automobile15.png
automobile/automobile1.png
automobile/automobile11

In [23]:
# %cd snifgsm
# !tar zcvf ../snifgsm.tgz *
# %cd ..

## Visualization

In [ ]:
# import matplotlib.pyplot as plt

# classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

# plt.figure(figsize=(10, 20))
# cnt = 0
# for i, cls_name in enumerate(classes):
#     path = f'{cls_name}/{cls_name}1.png'
#     # benign image
#     cnt += 1
#     plt.subplot(len(classes), 4, cnt)
#     im = Image.open(f'./data/{path}')
#     logit = model(transform(im).unsqueeze(0).to(device))[0]
#     predict = logit.argmax(-1).item()
#     prob = logit.softmax(-1)[predict].item()
#     plt.title(f'benign: {cls_name}1.png\n{classes[predict]}: {prob:.2%}')
#     plt.axis('off')
#     plt.imshow(np.array(im))
#     # adversarial image
#     cnt += 1
#     plt.subplot(len(classes), 4, cnt)
#     im = Image.open(f'./ifgsm/{path}')
#     logit = model(transform(im).unsqueeze(0).to(device))[0]
#     predict = logit.argmax(-1).item()
#     prob = logit.softmax(-1)[predict].item()
#     plt.title(f'adversarial: {cls_name}1.png\n{classes[predict]}: {prob:.2%}')
#     plt.axis('off')
#     plt.imshow(np.array(im))
# plt.tight_layout()
# plt.show()

## Report Question
* Make sure you follow below setup: the source model is "resnet110_cifar10", applying the vanilla fgsm attack on `dog2.png`. You can find the perturbed image in `fgsm/dog2.png`.

In [ ]:
# # original image
# path = f'dog/dog2.png'
# im = Image.open(f'./data/{path}')
# logit = model(transform(im).unsqueeze(0).to(device))[0]
# predict = logit.argmax(-1).item()
# prob = logit.softmax(-1)[predict].item()
# plt.title(f'benign: dog2.png\n{classes[predict]}: {prob:.2%}')
# plt.axis('off')
# plt.imshow(np.array(im))
# plt.tight_layout()
# plt.show()

# # adversarial image 
# adv_im = Image.open(f'./nifgsm/{path}')
# logit = model(transform(adv_im).unsqueeze(0).to(device))[0]
# predict = logit.argmax(-1).item()
# prob = logit.softmax(-1)[predict].item()
# plt.title(f'adversarial: dog2.png\n{classes[predict]}: {prob:.2%}')
# plt.axis('off')
# plt.imshow(np.array(adv_im))
# plt.tight_layout()
# plt.show()

## Passive Defense - JPEG compression
JPEG compression by imgaug package, compression rate set to 70

Reference: https://imgaug.readthedocs.io/en/latest/source/api_augmenters_arithmetic.html#imgaug.augmenters.arithmetic.JpegCompression

Note: If you haven't implemented the JPEG compression, this module will return an error. Don't worry about this.

In [ ]:
# import imgaug.augmenters as iaa

# # pre-process image
# x = transforms.ToTensor()(adv_im)*255
# x = x.permute(1, 2, 0).numpy()
# x = x.astype(np.uint8)

# # TODO: use "imgaug" package to perform JPEG compression (compression rate = 70)
# # compressed_x =  ... x .. 
# aug = iaa.JpegCompression(compression=70)
# compressed_x = aug.augment_image(x)
# logit = model(transform(compressed_x).unsqueeze(0).to(device))[0]
# predict = logit.argmax(-1).item()
# prob = logit.softmax(-1)[predict].item()
# plt.title(f'JPEG adversarial: dog2.png\n{classes[predict]}: {prob:.2%}')
# plt.axis('off')


# plt.imshow(compressed_x)
# plt.tight_layout()
# plt.show()